In [15]:
import gurobipy as gp
import pandas as pd
import ast

In [16]:
od_matrix = pd.read_csv('Data_SmallCase/Model_Input/od_matrix.csv', index_col=0)
canal_network = pd.read_csv('Data_SmallCase/Model_Input/canal_network.csv', index_col=0)
demand = pd.read_csv('Data_SmallCase/Model_Input/demand.csv')

# convert the column names to integers
od_matrix.columns = od_matrix.columns.astype(int)
canal_network.columns = canal_network.columns.astype(int)

od_matrix

,0,1,2,3
0,0,1,1,1
1,1,0,1,2
2,1,1,0,1
3,1,2,1,0


In [17]:
available_vessel = 3
tot_periods = int(len(demand) / len(od_matrix)) 

# basic sets
Z = od_matrix.index.tolist() # set of zones
Z_S = canal_network.index.tolist() # set of zones with a candidate stop
T = list(range(tot_periods)) # set of time periods

N = [(z,t) for z in Z for t in T] # set of nodes
N_S = [(z,t) for z in Z_S for t in T] # set of nodes where vessels could be located

In [18]:
##### for courier self rebalancing flows
# possible source nodes pointing to n
def gen_source_ns(n):
    possible_source_nodes = []
    for zone in Z:
        if zone == n[0] and n[1] != 0:
            possible_source_nodes.append((zone, n[1]-1))
        travel_time = od_matrix.iloc[zone, n[0]]
        if n[1]-travel_time >= 0:
            possible_source_nodes.append((zone, n[1]-travel_time))
    return possible_source_nodes

# possible sink nodes for n
def gen_sink_ns(n):
    reachable_nodes = []
    for zone in Z:
        if zone == n[0] and n[1] != tot_periods-1:
            reachable_nodes.append((zone, n[1]+1))
        travel_time = od_matrix.iloc[n[0], zone]
        if n[1]+travel_time <= tot_periods-1:
            reachable_nodes.append((zone, n[1]+travel_time))
    return reachable_nodes

# set of plus(n)
N_plus = {}
for n in N:
    N_plus[n] = gen_sink_ns(n)

# set of minus(n)
N_minus = {}
for n in N:
    N_minus[n] = gen_source_ns(n)

A = [(m, n) for n in N for m in N_minus[n]] # set of arcs
A1 = [(n,m) for n in N for m in N_plus[n]] # set of arcs

print([a for a in A if a not in A1])
print([a for a in A1 if a not in A])

[]
[]


In [19]:
def sink_vessel(w):
    linked_nodes = []
    # get zones with a candidate stop that is linked to the zone of w
    linked_phy_stops = canal_network.columns[canal_network.loc[w[0],:] == 1].tolist()
    for stop in linked_phy_stops + [w[0]]:
        if w[1]+1 <= tot_periods-1:
            linked_nodes.append((stop, w[1]+1))
    return linked_nodes

def source_vessel(w):
    linked_nodes = []
    # get zones with a candidate stop that is linked to the zone of w
    linked_phy_stops = canal_network.columns[canal_network.loc[w[0],:] == 1].tolist()
    for stop in linked_phy_stops + [w[0]]:
        if w[1]-1 >= 0:
            linked_nodes.append((stop, w[1]-1))
    return linked_nodes

NS_plus_route = {}
for w in N_S:
    NS_plus_route[w] = sink_vessel(w)

NS_minus_route = {}
for w in N_S:
    NS_minus_route[w] = source_vessel(w)

# set of vessel movement arcs
A_S = []
for w in N_S:
    for node in sink_vessel(w):
        A_S.append((w, node))

# # remove arcs that associated with the fictitious nodes
# A_S = [a for a in A_S if a[0] != 'start' and a[1] != 'end']

A_S1 = []
for w in N_S:
    for node in source_vessel(w):
        A_S1.append((node, w))

A_S1 = [a for a in A_S1 if a[0] != 'start' and a[1] != 'end']
print([a for a in A_S if a not in A_S1])
print([a for a in A_S1 if a not in A_S])


[]
[]


Routes generation

In [20]:
routes = []

candidate_routes = []
for i in [(0,t) for t in T]:
    candidate_route = []
    candidate_route.append(i)
    candidate_routes.append(candidate_route)

for i in N_S:
    for candi_route in candidate_routes:
        if candi_route[-1][0] == 0 and candi_route not in routes:
            routes.append(candi_route)
        if i == candi_route[-1]:
            for j in NS_plus_route[i]:
                new_route = candi_route.copy()
                new_route.append(j)
                candidate_routes.append(new_route)
                if j[0] == 0 and new_route not in routes:
                    routes.append(new_route)

                
# the set of routes
K = list(range(len(routes)))

# the set of vertices in a route
H = {}
for k in K:
    H[k] = routes[k]

# parameter returns whether a node is in a route
j = {}
for w in N_S:
    for k in K:
        j[w,k] = 1 if w in H[k] else 0

# parameter returns the length of a route
l = {}
for k in K:
    l[k] = len(routes[k])

# the subsequent node of a node in a route
def subsequent_node(n, k):
    for i in range(len(H[k])-1):
        if H[k][i] == n:
            return H[k][i+1]

# # check whether an arcs a is in a route k
# def is_arc_in_route(a, route):
#     if a in route:
#         return 1

# # parameter returns whether an arc is in a route
# f = {}
# for a in A_S:
#     for k in K:
#         f[a,k] = is_arc_in_route(a, routes[k])


## Model

In [21]:
model = gp.Model('BikeRebalancing_routeBased')

# 1 if a route is selected; otherwise, 0
theta = {}
for k in K:
    theta[k] = model.addVar(vtype=gp.GRB.BINARY, name=f'theta_{k}')

# 1 if a stop z ∈ Z_S is open; otherwise, 0
zeta = {}
for z in Z_S:
    zeta[z] = model.addVar(vtype=gp.GRB.BINARY, name=f'zeta_{z}')

# non-negative integer, number of dropped bikes to node n ∈ N from vessel v ∈ V at node w ∈ N_minus(n) cap N_S
dBar = {}
for k in K:
    for n in N:
        for w in [x for x in N_minus[n] if x in H[k]]:
            dBar[w,n,k] = model.addVar(vtype=gp.GRB.INTEGER, name=f'dBar_{w}_{n}_{k}')

# non-negative integer, number of collected bikes from node n ∈ N to vessel v ∈ V at node w ∈ N_plus(n) cap N_S
rBar = {}
for k in K:
    for n in N:
        for w in [x for x in N_plus[n] if x in H[k]]:
            rBar[n,w,k] = model.addVar(vtype=gp.GRB.INTEGER, name=f'rBar_{n}_{w}_{k}')

# non-negative integer, number of bikes self-redistributed by couriers to node n ∈ N from node m ∈ N_plus(n)
g = {}
for a in A:
    g[a] = model.addVar(vtype=gp.GRB.INTEGER, name=f'g_{a}')

# non-negative integer, number of bikes borrowed to node n ∈ N from w ∈ N_minus(n) cap N_S
b = {}
for n in N:
    for w in [x for x in N_minus[n] if x in N_S]:
        b[w,n] = model.addVar(vtype=gp.GRB.INTEGER, name=f'b_{w}_{n}')

#  non-negative integer, number of bikes returned from node n ∈ N to the stop-time node w ∈ N_plus(n) cap N_S
q = {}
for n in N:
    for w in [x for x in N_plus[n] if x in N_S]:
        q[n,w] = model.addVar(vtype=gp.GRB.INTEGER, name=f'q_{n}_{w}')

# non-negative integer, the inventory level of the stop associated with stop-time node w ∈ N_S
p = {}
for w in N_S:
    p[w] = model.addVar(vtype=gp.GRB.INTEGER, name=f'p_{w}')

# non-negative integer, inventory level of vessel v ∈ V at stop-period node ∈ N_S
iBar = {}
for k in K:
    for w in N_S:
        iBar[w,k] = model.addVar(vtype=gp.GRB.INTEGER, name=f'iBar_{w}_{k}')




In [22]:
# ---------------------demand ---------------------
# demand in each zone at each period
Demands = {}
for n in N:
    zone_id = n[0]
    period = n[1]
    # return the value in the column of "Rental" by zone_id and period in df demand
    Demands[n] = int(demand.loc[(demand['Zone_id'] == zone_id) & (demand['Period'] == period), 'Rental'].values[0])

# return requests in each stop at each period
Returns = {}
for n in N:
    zone_id = n[0]
    period = n[1]
    Returns[n] = int(demand.loc[(demand['Zone_id'] == zone_id) & (demand['Period'] == period), 'Return'].values[0])

# ---------------------vessel and stop capacity ---------------------
# vessel capacity
vessel_capacity = 50

# stop capacity
stop_capacity = 5


# ---------------------cost ---------------------
vessel_cost = 500 # euro/vessel, single vessel cost
vessel_operation_cost = 0.1 # euro/unit operation time
stop_cost = 300  # euro/stop, cost of openning a stop
bike_cost = 250  # euro/bike, cost of purchasing a bike
courier_reblc_cost = 7.5  # euro/bike/unit distance, cost of asking couriers to serve one rental or return demand covering one unit distance

BigM = 200


In [23]:
# quantity of rented vessels
route_quant = gp.quicksum(theta[k] for k in K)
vessel_operation_time = gp.quicksum(l[k] * theta[k] for k in K)
open_stop_quant = gp.quicksum(zeta[z] for z in Z_S)
bike_buy = gp.quicksum(iBar[H[k][0],k] for k in K) + gp.quicksum(p[(z,0)] for z in Z_S)
bikeDist_handby_courier = gp.quicksum(g[m,n] * od_matrix.loc[m[0],n[0]] for n in N for m in N_minus[n])


bike_droppedby_vessel = gp.quicksum(dBar[w,n,k] for k in K for n in N for w in [x for x in N_minus[n] if x in H[k]])
bike_pickupby_vessel = gp.quicksum(rBar[n,w,k] for k in K for n in N for w in [i for i in N_plus[n] if i in H[k]])
# return_satisfiedby_stop = gp.quicksum(q[n,w].X for n in N for w in [i for i in N_plus[n] if i in N_S])
# demand_satisfiedby_stop = gp.quicksum(b[w,n].X for n in N for w in [x for x in N_minus[n] if x in N_S])

flow_cost = 0.1 * (bike_droppedby_vessel + bike_pickupby_vessel)

# objective function
obj = vessel_cost * route_quant + vessel_operation_cost * vessel_operation_time + stop_cost * open_stop_quant + bike_cost * bike_buy + courier_reblc_cost * bikeDist_handby_courier + flow_cost
# obj = vessel_cost * vessel_quant + vessel_operation_cost * vessel_operation_time + stop_cost * open_stop_quant + courier_reblc_cost * bikeDist_handby_courier


model.setObjective(obj, gp.GRB.MINIMIZE)
model.update()

In [24]:
# maximum number of routes
con1 = model.addConstr(gp.quicksum(theta[k] for k in K) <= available_vessel, name='con1')

# stop selection
con2 = {}
for z in Z_S:
    con2[z] = model.addConstr(gp.quicksum(j[(z,t),k] * theta[k] for t in T for k in K) >= zeta[z], name=f'con2_{z}')

con2_1 = {}
for z in Z_S:
    con2_1[z] = model.addConstr(gp.quicksum(j[(z,t),k] * theta[k] for t in T for k in K) <= BigM * zeta[z], name=f'con2_1_{z}')

# bikes dropoff flow activation
con3 = {}
for k in K:
    for n in N:
        for w in [x for x in N_minus[n] if x in H[k]]:
            con3[w,n,k] = model.addConstr(dBar[w,n,k] <= BigM * theta[k] , name=f'con3_{w}_{n}_{k}')

# bikes pickup flow activation
con4 = {}
for k in K:
    for n in N:
        for w in [x for x in N_plus[n] if x in H[k]]:
            con4[n,w,k] = model.addConstr(rBar[n,w,k] <= BigM * theta[k] , name=f'con4_{n}_{w}_{k}')

# demand satisfaction
con5 = {}
for n in N:
    con5[n] = model.addConstr(gp.quicksum(dBar[w,n,k] for k in K for w in [x for x in N_minus[n] if x in H[k]]) + gp.quicksum(b[w,n] for w in [x for x in N_minus[n] if x in N_S]) + gp.quicksum(g[m,n] for m in N_minus[n])== Demands[n], name=f'con5_{n}')

# return satisfaction
con6 = {}
for n in N:
    con6[n] = model.addConstr(gp.quicksum(rBar[n,w,k] for k in K for w in [x for x in N_plus[n] if x in H[k]]) + gp.quicksum(q[n,w] for w in [x for x in N_plus[n] if x in N_S]) + gp.quicksum(g[n,m] for m in N_plus[n])== Returns[n], name=f'con6_{n}')

# vessel inventory balance
con7 = {}
for k in K:
    for w in H[k][:-1]:
        con7[w,k] = model.addConstr(iBar[subsequent_node(w,k),k] == iBar[w,k] + gp.quicksum(rBar[n,w,k] for n in N_minus[w]) - gp.quicksum(dBar[w,n,k] for n in N_plus[w]), name=f'con7_{w}_{k}')
    
# vessel in ventory capacity
con8 = {}
for k in K:
    for w in H[k]:
        con8[w,k] = model.addConstr(iBar[w,k] <= vessel_capacity * theta[k], name=f'con8_{w}_{k}')

con8_1 = {}
for k in K:
    for w in H[k][-1:]:
        con8_1[w,k] = model.addConstr(iBar[w,k] + gp.quicksum(rBar[n,w,k] for n in N_minus[w]) - gp.quicksum(dBar[w,n,k] for n in N_plus[w]) <= vessel_capacity * theta[k] , name=f'con8_1_{w}_{k}')

con8_2 = {}
for k in K:
    for w in H[k][-1:]:
        con8_2[w,k] = model.addConstr(iBar[w,k] + gp.quicksum(rBar[n,w,k] for n in N_minus[w]) - gp.quicksum(dBar[w,n,k] for n in N_plus[w]) >= 0 , name=f'con8_2_{w}_{k}')

# stop inventory tracking
con9 = {}
for z in Z_S:
    for t in T[:-1]:
        con9[(z,t)] = model.addConstr(p[(z,t+1)] == p[(z,t)] + gp.quicksum(q[n,(z,t)] for n in N_minus[(z,t)]) - gp.quicksum(b[(z,t),n] for n in N_plus[(z,t)]), name=f'con9_{z}_{t}')
    for t in T[-1:]:
        con9[(z,t)] = model.addConstr(p[(z,t)] + gp.quicksum(q[n,(z,t)] for n in N_minus[(z,t)]) - gp.quicksum(b[(z,t),n] for n in N_plus[(z,t)]) - stop_capacity * zeta[w[0]] <= 0, name=f'con9_{z}_{t}')

# stop service capability 
con10 = {}
for w in N_S:
    con10[w] = model.addConstr(gp.quicksum(q[n,w] for n in N_minus[w]) + gp.quicksum(b[w,n] for n in N_plus[w]) - BigM * zeta[w[0]] <= 0, name=f'con10_{w}')

# stop capacity
con11 = {}
for w in N_S:
    con11[w] = model.addConstr(p[w] - stop_capacity * zeta[w[0]] <= 0, name=f'con11_{w}')


model.write("cosntrains-route.lp")

In [25]:
# optimize the model
model.optimize()

# model.computeIIS()
# model.write("model.ilp")

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 427 rows, 546 columns and 1804 nonzeros
Model fingerprint: 0xf60d803e
Variable types: 0 continuous, 546 integer (16 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [1e-01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 17113.200000
Presolve removed 281 rows and 391 columns
Presolve time: 0.00s
Presolved: 146 rows, 155 columns, 547 nonzeros
Variable types: 0 continuous, 155 integer (45 binary)
Found heuristic solution: objective 17112.300000

Root relaxation: objective 1.586730e+04, 163 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  

In [26]:
# print the objective value
print(f"Objective value = {model.objVal}")

all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)

for name, val in zip(names, values):
    if val != 0:
        print(f"{name} = {val}")

Objective value = 16110.4
theta_12 = 1.0
zeta_0 = 1.0
zeta_2 = 1.0
dBar_(0, 0)_(0, 1)_12 = 13.0
dBar_(0, 0)_(1, 1)_12 = 15.0
dBar_(0, 0)_(2, 1)_12 = 6.0
dBar_(0, 0)_(3, 1)_12 = 16.0
rBar_(0, 2)_(0, 3)_12 = 14.0
rBar_(1, 2)_(0, 3)_12 = 15.0
rBar_(2, 2)_(0, 3)_12 = 15.0
rBar_(3, 2)_(0, 3)_12 = 6.0
g_((0, 1), (0, 1)) = 1.0
b_(0, 0)_(0, 0) = 1.0
b_(0, 0)_(2, 1) = 4.0
b_(2, 1)_(2, 1) = 5.0
q_(3, 2)_(0, 3) = 5.0
q_(3, 2)_(2, 3) = 5.0
p_(0, 0) = 5.0
p_(2, 0) = 5.0
p_(2, 1) = 5.0
iBar_(0, 0)_12 = 50.0


In [27]:
vessel_quant_sol = sum(theta[k].X for k in K)
vessel_operation_time_sol = sum(l[k] * theta[k].X for k in K)
open_stop_quant_sol = gp.quicksum(zeta[z].X for z in Z_S)
bike_buy_sol = sum(iBar[H[k][0],k].X for k in K) + gp.quicksum(p[(z,0)].X for z in Z_S)
bikeDist_handby_courier_sol = sum(g[m,n].X for n in N for m in N_minus[n])

bike_droppedby_vessel = sum(dBar[w,n,k].X for k in K for n in N for w in [x for x in N_minus[n] if x in H[k]])
bike_pickupby_vessel = sum(rBar[n,w,k].X for k in K for n in N for w in [i for i in N_plus[n] if i in H[k]])
return_satisfiedby_stop = sum(q[n,w].X for n in N for w in [i for i in N_plus[n] if i in N_S])
demand_satisfiedby_stop = sum(b[w,n].X for n in N for w in [x for x in N_minus[n] if x in N_S])
self_rbl_flow = sum(g[a].X for a in A)

print(f"Total vessel used: {vessel_quant_sol}")
print(f"Total bike purchased: {bike_buy_sol}")
print(f"Total stop opened: {open_stop_quant_sol}")
print(f"Total bike dropped by vessel: {bike_droppedby_vessel}")
print(f"Total bike pickup by vessel: {bike_pickupby_vessel}")
print(f"Total demand satisfied by stop: {demand_satisfiedby_stop}")
print(f"Total return satisfied by stop: {return_satisfiedby_stop}")
print(f"Total bike self redistributed by couriers: {self_rbl_flow}")

Total vessel used: 1.0
Total bike purchased: 60.0
Total stop opened: 2.0
Total bike dropped by vessel: 50.0
Total bike pickup by vessel: 50.0
Total demand satisfied by stop: 10.0
Total return satisfied by stop: 10.0
Total bike self redistributed by couriers: 1.0


In [28]:
H[12]

[(0, 0), (0, 1), (2, 2), (0, 3)]